In [2]:
import requests
import pandas as pd

In [3]:
def get_characters(cmtitle, cmcontinue=None):
    print(f"Getting {cmtitle}")
    url = "https://starwars.fandom.com/api.php"
    params = {
        "action": "query",
        "list": "categorymembers",
        # "cmtitle": "Category:Clan of the Toribota members",
        "cmtitle": cmtitle,
        "cmlimit": 500,
        "format": "json"
    }
    if cmcontinue:
        params["cmcontinue"] = cmcontinue
    
    response = requests.get(url, params=params)
    data = response.json()
    
    members = data["query"]["categorymembers"]
    names = [m["title"] for m in members]
    
    continue_token = data.get("continue", {}).get("cmcontinue")
    
    return names, continue_token

In [4]:
def get_until_continue_stops(cmtitle):
    all_names = []
    continue_token = None
    while True:
        names, continue_token = get_characters(cmtitle, cmcontinue=continue_token)
        all_names.extend(names)
        print(f"Got {len(names)} names, total: {len(all_names)}")
        if not continue_token:
            break
    return all_names

In [4]:
# This kills the kernel. DO NOT USE!
def recursive_grab_all(cmtitle, prefix=""):
    unfiltered_names = get_until_continue_stops(cmtitle)
    intermediate_names = []
    for name in unfiltered_names:
        if name[:9] == "Category:":
            new_names = recursive_grab_all(name, prefix=f"{prefix}/{name.replace(' ', '_')}")
            intermediate_names += intermediate_names
        else:
            intermediate_names.append(name)
    return_names = [f"{prefix}: {name}" for name in intermediate_names]
    return return_names

In [25]:
names = []
undone_categories = ["Category:Individuals"]
done_categories = []

while len(undone_categories) > 0:
    print(f"Undone: {len(undone_categories)}, done: {len(done_categories)}, names: {len(names)}")
    cmtitle = undone_categories[0]
    category_contents = get_until_continue_stops(cmtitle)
    for content in category_contents:
        if content[:9] == "Category:":
            if content not in done_categories and content not in undone_categories:
                # I suspect there's circular reference here...
                undone_categories.append(content)
        else:
            names.append(content)
    done_categories.append(cmtitle)
    undone_categories.remove(cmtitle)

Undone: 1, done: 0, names: 0
Getting Category:Individuals
Got 43 names, total: 43
Undone: 24, done: 1, names: 19
Getting Category:Individuals by faction
Got 177 names, total: 177
Undone: 200, done: 2, names: 19
Getting Category:Individuals by family
Got 110 names, total: 110
Undone: 309, done: 3, names: 19
Getting Category:Individuals by gender
Got 8 names, total: 8
Undone: 316, done: 4, names: 19
Getting Category:Individuals by occupation
Got 102 names, total: 102
Undone: 416, done: 5, names: 19
Getting Category:Individuals by planet
Got 246 names, total: 246
Undone: 660, done: 6, names: 19
Getting Category:Individuals by pronouns
Got 7 names, total: 7
Undone: 666, done: 7, names: 19
Getting Category:Individuals by species
Got 500 names, total: 500
Getting Category:Individuals by species
Got 500 names, total: 1000
Getting Category:Individuals by species
Got 68 names, total: 1068
Undone: 1730, done: 8, names: 19
Getting Category:Castaways
Got 52 names, total: 52
Undone: 1729, done: 9, 

In [29]:
pd.DataFrame({"name": names}).drop_duplicates().to_csv("datasets/full_sw_names.csv", index=False)